## Library Imports and Variable Declarations:

In [ ]:
import numpy as np
from PIL import Image
import os
import random
import copy

In [ ]:
Z = 17 # (Input)
C = 10 # (Input)
M = N = 60 # image size (Input)

K = 0 # Stores the total number of images used
D = M ** 2 # Dimension of a flattened image

# Celeb dictionary to use for class creation
celeb_dict = {"Angelina Jolie" : 1, "Brad Pitt" : 2, "Denzel Washington" : 3, "Hugh Jackman" : 4, "Jennifer Lawrence" : 5, "Johnny Depp" : 6, "Kate Winslet" : 7, "Leonardo DiCaprio" : 8, "Megan Fox" : 9, "Natalie Portman" : 10, "Nicole Kidman" : 11, "Robert Downey Jr" : 12, "Sandra Bullock" : 13, "Scarlett Johansson" : 14, "Tom Cruise" : 15, "Tom Hanks" : 16, "Will Smith" : 17}

## Image Matrix Construction:

In [ ]:
location = '/content/drive/MyDrive/Cropped Celebrity Faces Dataset' # (Input)

Image_Mat = np.zeros((D,1)) # Default numpy array
Celeb_Name = []

# Iterating through the directory of folders (celeb names)
for celeb in os.listdir(location):
    celeb_path = os.path.join(location, celeb)
    if not os.path.isdir(celeb_path):
        continue  # Skip if the item in the directory is not a folder

    files = [f for f in os.listdir(celeb_path) if os.path.isfile(os.path.join(celeb_path, f))]

    # Use min() to avoid going beyond the available indices
    selected_files = random.sample(files, min(len(files), C))  # Unbalanced dataset alert!!

    for selected_file in selected_files:
        Celeb_Name.append(celeb_dict[celeb]) # Form the Y train dataset

        # Open, resize and append the face image
        face_path = os.path.join(celeb_path, selected_file)
        face_image = Image.open(face_path).convert("L")  # Convert to grayscale
        resized_face_image = face_image.resize((M, N))
        # Row-major flattening and convertion into column vector
        img_array = np.array(resized_face_image).flatten().reshape((D,1))
        Image_Mat = np.append(Image_Mat, img_array, axis=1)

        K+=1 # Increment the counter for total number of images

Image_Mat = Image_Mat[:,1:]
Celeb_Name = np.array(Celeb_Name)

In [ ]:
# def view(A,size,n):
#   '''
#   Display the 'n'-th image (column) from the 'A' matrix

#   n: number less than len(A)
#   A: Matrix of Image Column-Wise
#   '''
#   reshaped_image = A[:,n].reshape(size,size)
#   # Convert the reshaped image to a PIL Image
#   image = Image.fromarray(reshaped_image.astype('uint8'))
#   display(image)

# val=6
# view(Image_Mat,M,val)
# print(Celeb_Name[val])

## Computation of Class-Specific Mean Vectors:

In [ ]:
mean_class_vectors = [] # List of Class-Specific Mean Vectors
class_matrices = [] # List of Class Matrices

for i in range(0,Z):
  class_matrices.append(np.copy(Image_Mat[:, i*C:(i+1)*C]))
  mean_class_vectors.append(np.mean(class_matrices[i], axis=1, keepdims=True))


## Computation of Global Mean Vector:

In [ ]:
mean_overall = np.mean(Image_Mat, axis=1, keepdims=True)
mean_overall.shape

(3600, 1)

## Within-Class Scatter Matrix (S w) Construction:

In [ ]:
Sw = np.zeros((D,D))

for i in range(0,Z):
  temp = (class_matrices[i] - mean_class_vectors[i])
  Sw += temp @ temp.T

Sw.shape

(3600, 3600)

## Between-Class Scatter Matrix (S b) Construction:

In [ ]:
Sb = np.zeros((D,D))

for i in range(0,Z):
  temp = mean_class_vectors[i] - mean_overall
  Sb += temp @ temp.T

Sb *= C
Sb.shape

(3600, 3600)

## Eigenvalue and Eigenvector Computation:

In [ ]:
# Calculate the product: Sw-1Sb
A = np.linalg.inv(Sw) @ Sb

eigenValues, eigenVectors = np.linalg.eig(A)
eigenValues = np.real(eigenValues)

In [ ]:
idx = eigenValues.argsort()[::-1]
eigenValues = eigenValues[idx]
eigenVectors = eigenVectors[:, idx]

In [ ]:
S = 24
U = np.copy(eigenVectors[:,:S])

## Projection onto S-dimensional subspace:

In [ ]:
Y = U.T @ Image_Mat